# Weiss Schwarz Damage Simulation
Monte Carlo simulation of damage passing through, including trial count tuning and histogram output.

In [ ]:
from ws_sim.monte_carlo import DeckConfig, simulate_trials, cumulative_probability_at_least, tune_trial_count
from ws_sim.plotting import plot_cumulative_histogram
import matplotlib.pyplot as plt

damage_sequence = [2, 3, 3]
deck = DeckConfig(total_cards=50, climax_cards=8)

chosen_trials, history = tune_trial_count(damage_sequence, deck, threshold=6, target_error=0.02, min_trials=500, seed=42)
damages = simulate_trials(damage_sequence, deck, trials=chosen_trials, seed=42)
thresholds = range(0, max(damages) + 1)
probabilities = cumulative_probability_at_least(damages, thresholds)

fig, ax = plot_cumulative_histogram(probabilities)
ax.text(0.99, 0.95, f'trials={chosen_trials}', transform=ax.transAxes, ha='right', va='top')
plt.show()
fig.savefig('../artifacts/damage_hist.png', bbox_inches='tight')

## Simulating mid-sequence refreshes with a preloaded waiting room

Use `DeckConfig`'s `waiting_room_cards` and `waiting_room_climax_cards` to model game states where cards have already been routed to the waiting room before you start a damage sequence. The simulator will draw from the reduced deck size, shuffle the waiting room back when the deck empties, and automatically apply the one-damage refresh penalty whenever that shuffle happens mid-sequence. This makes it easy to study lines that intentionally compress the deck and to see how refresh timing impacts the damage curve.


In [ ]:
realistic_damage_sequence = [3, 2, 4, 3, 3, 4]

deck_with_waiting_room = DeckConfig(
    total_cards=50,
    climax_cards=8,
    waiting_room_cards=30,
    waiting_room_climax_cards=3,
)

mid_refresh_threshold = 7

chosen_trials_wr, history_wr = tune_trial_count(
    realistic_damage_sequence,
    deck_with_waiting_room,
    threshold=mid_refresh_threshold,
    target_error=0.02,
    min_trials=500,
    seed=7,
)

damages_with_refresh = simulate_trials(
    realistic_damage_sequence,
    deck_with_waiting_room,
    trials=chosen_trials_wr,
    seed=7,
)

thresholds_wr = range(0, max(damages_with_refresh) + 1)
probabilities_wr = cumulative_probability_at_least(damages_with_refresh, thresholds_wr)

fig_wr, ax_wr = plot_cumulative_histogram(probabilities_wr)
ax_wr.set_title("Damage with preloaded waiting room (refresh penalty included)")
ax_wr.text(0.99, 0.95, f'trials={chosen_trials_wr}', transform=ax_wr.transAxes, ha='right', va='top')
plt.show()

fig_wr.savefig('../artifacts/damage_hist_waiting_room.png', bbox_inches='tight')
